In [ ]:
import pandas as pd
import random

df = pd.read_csv("patient_data.csv")
n = len(df.index)

df.head()

Cleaning the `notes` column of the dataframe.

In [ ]:
# def clean_str(input):
#     input = input.split()
#     input = " ".join(input).strip()
#     return input

# df["notes"] = df["notes"].apply(clean_str)

# # Assuming df is your DataFrame
# with open(f"anonymized_patient_notes.txt", 'w', encoding='utf-8') as file:
#     for _, row in df.iterrows():
#         file.write(row['notes'])

In [ ]:
import requests
from pprint import pprint
import json


def ollama_request(prompt, model="hermes"):
    url = "http://localhost:11434/api/generate"
    param = {"model": "hermes", "prompt": prompt, "stream": False, "raw": True}
    res = requests.post(url, json=param).json()
    pprint(res)
    bot_response = res["response"]
    sec = res["eval_duration"] / 1000000000
    tok_s = res["eval_count"] / sec
    return bot_response


def obtain_qa(note, model="hermes"):
    q_prompt = f'MEDICAL NOTE: """\n{note}""" \nBased on the given medical note, what is be the single most probably inquiry or question the patient asked to the doctor? '
    q = ollama_request(q_prompt, model)

    ans_prompt = f'PATIENT QUESTION:  """\n{q}""" \n MEDICAL NOTE: """\n{note}""" \nBased on the given medical note and patient question, construct a concise and terse paragraph of a top professional doctors response in 3 to 4 sentences. '
    ans = ollama_request(ans_prompt)

    return q, ans


def obtain_qa_single_run(note, model="hermes"):
    prompt = f'MEDICAL NOTE: """\n{note}""" \nBased on the given medical note, construct one `Question` and `Answer` pair between the patient and the doctor in JSON format with exactly one pair of `Question` and `Answer`. The patients question includes clear and detailed description of the problem relevant in the medical note. The doctors answer is in first person perspective, and includes reasoning and details, such as sympotoms, diagnosis, inference, suggestions, medications. Both questions and answers should be concise, straight to the point and highly medically relevant. '

    res = ollama_request(prompt, model)
    return res


r = random.randint(0, n)

sample_note = df.iloc[r].notes

# q, ans = obtain_qa(sample_note)
# print(f"==== question: {q}")
# print(f"==== answer: {ans}")

print(f"\n\n {obtain_qa_single_run(sample_note, 'neural')}")